In [1]:
from pathlib import Path
from PIL import Image

import os
import json
import pandas as pd
import torchvision, torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from datetime import datetime

In [4]:
def dataframe(path, image):
    file_path = []
    if image:
        for root, dirs, files in os.walk(path):
            for file in files:
                a = os.path.join(root, file)
                file_path.append(a)
        df = pd.DataFrame(file_path, columns=["filepath"])
        return df
    else:
        for root, dirs, files in os.walk(path):
            for file in files:
                a = os.path.join(root, file)
                file_path.append(a)
        df = pd.DataFrame(file_path, columns=["filepath"])

        csv_data = []
        for index, row in df.iterrows():
            file_path = row["filepath"]
            print(file_path)
            if os.path.exists(file_path):  # 파일이 존재하는지 확인
                with open(file_path, 'r') as file:
                    file_content = json.load(file)
                    # JSON 데이터를 각각의 컬럼으로 분리하여 추가
                    csv_data.append([
                        file_path,
                        file_content.get("filename", ""),
                        file_content.get("id", ""),
                        file_content.get("gender", ""),
                        file_content.get("birth", ""),
                        file_content.get("annotation", "")
                    ])
            else:
                print(f"파일 '{file_path}'이(가) 존재하지 않습니다.")
        df_read = pd.DataFrame(csv_data, columns=[
            "File_Path", "Filename", "ID", "Gender", "Birth", "annotation"
        ])
        return df_read

In [5]:
# 경로설정
trn_img_path = Path("/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/데이터/train")
trn_label_path = Path("/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train")

tst_img_path = Path("/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/데이터/test")
tst_label_path = Path("/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/test")

# df 생성
df_trn_img = dataframe(trn_img_path, image=True)
df_trn_label = dataframe(trn_label_path, image=False)

df_tst_img = dataframe(tst_img_path, image=True)
df_tst_label = dataframe(tst_label_path, image=False)

/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_hat_distance1_00000341.json
/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_hat_distance2_00000132.json
/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_mask_distance1_00000224.json
/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_mask_distance2_00000258.json
/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_normal_distance1_00000183.json
/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_normal_distance1_00000310.json
/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_normal_distance1_00000318.json
/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_outdoor_back_mask_distance1_00000011.json
/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_outdoor_back_mask_distance1_00000296.json
/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_outdoor_back_mask_distance2_00000000.json
/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_outdoor_back_mask_distan

In [6]:
df_trn_label.head()

,File_Path,Filename,ID,Gender,Birth,annotation
0,/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_hat_distance1_00000341.json,2879_indoor_hat_distance1_00000341,2879,male,1983,"[{'box': {'x': 636.9613512169449, 'y': 326.7717516775838, 'w': 397.52374973517226, 'h': 464.36978242880286}, 'landmark': [[658.9501953125, 534.0327758789062], [659.122717846511, 430.5617174069092]..."
1,/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_hat_distance2_00000132.json,2879_indoor_hat_distance2_00000132,2879,male,1983,"[{'box': {'x': 625.7258925439521, 'y': 543.9121778581303, 'w': 315.94705393641783, 'h': 307.2453482676107}, 'landmark': [[672.9346951895866, 720.2760009765625], [655.7573646741444, 607.71234130859..."
2,/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_mask_distance1_00000224.json,2879_indoor_mask_distance1_00000224,2879,male,1983,"[{'box': {'x': 707.5109298036161, 'y': 387.7717516775839, 'w': 380.47066663128265, 'h': 411.3695332660562}, 'landmark': [[742.0310073424672, 567.0795883606578], [755.15185546875, 452.882080078125]..."
3,/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_mask_distance2_00000258.json,2879_indoor_mask_distance2_00000258,2879,male,1983,"[{'box': {'x': 722.3743994700883, 'y': 408.617491098738, 'w': 325.7369413990739, 'h': 352.9049596835523}, 'landmark': [[759.7973406568276, 544.7793242065627], [769.263671875, 459.5054626464844], [..."
4,/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_normal_distance1_00000183.json,2879_indoor_normal_distance1_00000183,2879,male,1983,"[{'box': {'x': 630.4688021157028, 'y': 348.00627301013355, 'w': 426.19909875496404, 'h': 469.78039555801024}, 'landmark': [[666.8556518554688, 567.8073120117188], [672.7732418236346, 434.885245809..."


In [7]:
def labelmodi(df):
    # 성별
    gender_map = {'male': 0, 'female': 1}
    df['Gender'] = df['Gender'].map(gender_map)


    #나이
    current_year = datetime.now().year

    # birth 열의 값을 현재 년도와 빼서 나이를 계산합니다.
    df['realage'] = current_year - df['Birth'].astype(int)

    # 나이를 기반으로 10대, 20대, 30대 등의 범주로 변환합니다.
    def categorize_age(age):
        if 10 <= age < 20:
            return '1'
        elif 20 <= age < 30:
            return '2'
        elif 30 <= age < 40:
            return '3'
        elif 40 <= age < 50:
            return '4'
        elif 50 <= age < 60:
            return '5'
        elif 60 <= age < 70:
            return '6'
        elif 70 <= age < 80:
            return '8'
        elif 80 <= age < 90:
            return '9'

    df['age'] = df['realage'].apply(categorize_age)
    drop_col = ["Birth", "realage"]
    df.drop(drop_col, axis=1, inplace=True)
    return df

In [8]:
df_trn_label = labelmodi(df_trn_label)
df_tst_label = labelmodi(df_tst_label)

In [12]:
# df_trn_label.head()

In [ ]:
#df_tst_label.haed()

In [16]:
train_img = df_trn_img.to_csv("./train_img.csv", index=False)
test_img = df_tst_img.to_csv("./test_img.csv", index=False)
train_label = df_trn_label.to_csv("./train_label.csv", index=False)
test_label = df_tst_label.to_csv("./test_label.csv", index=False)

In [18]:
df_mlabels = pd.read_csv("./train_label.csv")
df_mlabels.head()

,File_Path,Filename,ID,Gender,annotation,age
0,/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_hat_distance1_00000341.json,2879_indoor_hat_distance1_00000341,2879,0,"[{'box': {'x': 636.9613512169449, 'y': 326.7717516775838, 'w': 397.52374973517226, 'h': 464.36978242880286}, 'landmark': [[658.9501953125, 534.0327758789062], [659.122717846511, 430.5617174069092]...",4
1,/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_hat_distance2_00000132.json,2879_indoor_hat_distance2_00000132,2879,0,"[{'box': {'x': 625.7258925439521, 'y': 543.9121778581303, 'w': 315.94705393641783, 'h': 307.2453482676107}, 'landmark': [[672.9346951895866, 720.2760009765625], [655.7573646741444, 607.71234130859...",4
2,/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_mask_distance1_00000224.json,2879_indoor_mask_distance1_00000224,2879,0,"[{'box': {'x': 707.5109298036161, 'y': 387.7717516775839, 'w': 380.47066663128265, 'h': 411.3695332660562}, 'landmark': [[742.0310073424672, 567.0795883606578], [755.15185546875, 452.882080078125]...",4
3,/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_mask_distance2_00000258.json,2879_indoor_mask_distance2_00000258,2879,0,"[{'box': {'x': 722.3743994700883, 'y': 408.617491098738, 'w': 325.7369413990739, 'h': 352.9049596835523}, 'landmark': [[759.7973406568276, 544.7793242065627], [769.263671875, 459.5054626464844], [...",4
4,/mnt/d/Data/119.IR 실제 환경 안면 검출 및 인식 데이터/라벨/train/2879_indoor_normal_distance1_00000183.json,2879_indoor_normal_distance1_00000183,2879,0,"[{'box': {'x': 630.4688021157028, 'y': 348.00627301013355, 'w': 426.19909875496404, 'h': 469.78039555801024}, 'landmark': [[666.8556518554688, 567.8073120117188], [672.7732418236346, 434.885245809...",4
